# 0.0 Importando as bibliotecas e coletando os dados

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

In [14]:
path = 'C:/Users/allan/Documents/GitHub/python_ds_ao_dev/etl/'
database_name = 'database_hm.sqlite'

In [15]:
conn = create_engine('sqlite:///' + path + database_name)

In [16]:
query = "SELECT * FROM showroom"

In [17]:
df_raw = pd.read_sql(query, conn)

In [30]:
df_raw.head()

,product_id,product_name,product_type,product_colors,product_price,cotton,spandex,elastomultiester,polyester,lyocell,rayon,scrapy_datetime
0,1024256001,slim_jeans,men_jeans_slim,black,24.99,0.99,0.01,0.0,0.0,0.0,0.0,2023-01-10 21:00:39.000000
1,1024256001,slim_jeans,men_jeans_slim,light_denim_blue,24.99,0.99,0.01,0.0,0.0,0.0,0.0,2023-01-10 21:00:39.000000
2,1024256001,slim_jeans,men_jeans_slim,denim_blue,24.99,0.99,0.01,0.0,0.0,0.0,0.0,2023-01-10 21:00:39.000000
3,1024256001,slim_jeans,men_jeans_slim,dark_blue,24.99,0.99,0.01,0.0,0.0,0.0,0.0,2023-01-10 21:00:39.000000
4,1024256001,slim_jeans,men_jeans_slim,dark_denim_blue,24.99,0.99,0.01,0.0,0.0,0.0,0.0,2023-01-10 21:00:39.000000


# 1.0 Descrição dos dados

In [19]:
df01 = df_raw.copy()

### 1.1 Dimensão dos dados

In [21]:
shape = df01.shape
print(f'Number of rows: {shape[0]}')
print(f'Number of columns: {shape[1]}')

Number of rows: 722
Number of columns: 12


### 1.2 Tipos dos dados

In [22]:
df01.dtypes

product_id           object
product_name         object
product_type         object
product_colors       object
product_price       float64
cotton              float64
spandex             float64
elastomultiester    float64
polyester           float64
lyocell             float64
rayon               float64
scrapy_datetime      object
dtype: object

In [23]:
#convertendo scrapy_datetime para o formato de datas do pandas
df01["scrapy_datetime"] = pd.to_datetime(df01["scrapy_datetime"])

In [31]:
df01.dtypes

product_id                  object
product_name                object
product_type                object
product_colors              object
product_price              float64
cotton                     float64
spandex                    float64
elastomultiester           float64
polyester                  float64
lyocell                    float64
rayon                      float64
scrapy_datetime     datetime64[ns]
dtype: object

### 1.3 Identificação dos dados faltantes

In [25]:
df01.isna().sum()

product_id          0
product_name        0
product_type        0
product_colors      0
product_price       0
cotton              0
spandex             0
elastomultiester    0
polyester           0
lyocell             0
rayon               0
scrapy_datetime     0
dtype: int64

### 1.4 Substituição de dados faltantes

Não existem dados faltantes.

### 1.5 Descrição dos dados

In [32]:
num_attributes = df01.select_dtypes(include=["float64"])
cat_attributes = df01.select_dtypes(include=["object"])

#### 1.5.1 Colunas numéricas

In [34]:
num_attributes.head()

,product_price,cotton,spandex,elastomultiester,polyester,lyocell,rayon
0,24.99,0.99,0.01,0.0,0.0,0.0,0.0
1,24.99,0.99,0.01,0.0,0.0,0.0,0.0
2,24.99,0.99,0.01,0.0,0.0,0.0,0.0
3,24.99,0.99,0.01,0.0,0.0,0.0,0.0
4,24.99,0.99,0.01,0.0,0.0,0.0,0.0


In [67]:
#métricas de tendência central - media, mediana
central_tendency = num_attributes.describe().loc[["mean", "50%"]]
central_tendency = central_tendency.rename(index={'50%': 'median'})

#métricas de dispersão - std, min, max, range, skew, kurtosis
dispersion = num_attributes.describe().loc[["std", "min", "max"]]
d1 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T.rename(index={0:'range'})
d2 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T.rename(index={0:'skew'})
d3 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T.rename(index={0:'kurtosis'})
dispersion = pd.concat([dispersion, d1, d2, d3])

#concat
describe = pd.concat([dispersion, central_tendency]).T

describe

,std,min,max,range,skew,kurtosis,mean,median
product_price,7.100216,19.99,49.99,30.00,-0.300819,-0.293043,35.550942,34.99
cotton,0.071170,0.00,1.00,1.00,-8.047276,96.558294,0.971163,0.99
spandex,0.007510,0.00,0.02,0.02,-0.189185,-1.207719,0.011136,0.01
elastomultiester,0.019438,0.00,0.09,0.09,3.803933,12.567739,0.004820,0.00
polyester,0.044210,0.00,0.21,0.21,4.157907,15.369454,0.010111,0.00
lyocell,0.028927,0.00,0.55,0.55,18.960375,358.488877,0.001524,0.00
rayon,0.007889,0.00,0.15,0.15,18.960375,358.488877,0.000416,0.00


* <b> range </b>: estimativa de variabilidade ou dispersão.


* <b> média </b>: é um número que representa um ponto central de uma nuvem de pontos. Mais sensível aos outliers.


* <b> mediana </b>: é o número do meio de uma lista ordenada. Outra forma de resumir essa nuvem de pontos. Menos sensível aos outliers.


* <b> std </b>: medida de variabilidade/dispersão. A localização é uma forma de resumir uma nuvem de pontos. Mas é possível complementar ela. Medida de localização diz qual é o ponto central, medida de dispersão diz como os dados estão distribuídos em torno desse ponto central. É a raiz quadradada da variância. A variância é a soma dos desvios (de cada ponto a media) elevados ao quadrado (para remover o sinal -) dividido por n-1. Divide por n-1 para utilizar um estimador não enviesado, grau de liberdade = 1.


* <b> skewness </b>: medida do quanto uma distribuição de dados desvia de uma distribuição normal. Sendo que a distribuição normal não possui skewness, ou seja, os dados estão distribuídos perfeitamente ao redor da média.

    * Skewness positiva e negativa - para determinar se uma distribuição tem skewness positiva ou negativa é preciso olhar para duas métricas: a média e a mediana. Se média = mediana -> distribuição dos dados está simétrica, 0 skewness. Se mediana > média -> deslocamento dos dados para a direita, skewness negativa (cauda a esquerda). Se mediana < média -> deslocamento dos dados para a esquerda, skewness positiva (cauda a direita).

    * Como interpretar a skewness nos dados? Se tem skewness negativa (left-skewed), os dados estão concentrados acima da média. Se tem skewness positiva (right-skewed), os dados estão concentrados abaixo da média. É importante, também, saber a skewness da variável resposta, pois a maioria dos algoritmos de ML leva em conta que a distribuição é normal, pois se tem concentração dos dados de um lado ou de outro, está enviesando o modelo, pois terá mais exemplos de treino.


* <b> kurtosis </b> - descreve a distribuição dos dados em torno da média. Evidencia uma distribuição de caudas longas. (Probabilidade baixa de acontecer valores extremos).

    * Se não tem caudas longas, a distribuição é mais flat
    * kurtosis baixa -> não tem tanta cauda longa, distribuição mais flat, não tem concentração dos dados na média, tem mais chances de acontecer valores mais extremos tanto abaixo quanto na média
    * kurtosis alta -> tem cauda longa, concentração muito alta ao redor da média

#### 1.5.2 Colunas categóricas

# 2.0 Feature Engineering

# 3.0 Filtragem dos atributos

# 4.0 Exploratory Data Analysis